## Рассчет среднего и стандартного отклонения по частоте в каждом из контрольных образцов


In [24]:
import numpy as np
import pandas as pd

In [25]:
dir_control1 = "/Users/valeriaistuganova/Desktop/BioInf/project2/BioProject2/Control1/"
dir_control2 = "/Users/valeriaistuganova/Desktop/BioInf/project2/BioProject2/Control2/"
dir_control3 = "/Users/valeriaistuganova/Desktop/BioInf/project2/BioProject2/Control3/"
dir_sample = "/Users/valeriaistuganova/Desktop/BioInf/project2/BioProject2/Sample/"

## Читаем файл

In [26]:
from Parsing_vcf import read_vcf

control1 = read_vcf(dir_control1 + "VarScan_SRR1705858_results.vcf")
control2 = read_vcf(dir_control2 + "VarScan_SRR1705859_results.vcf")
control3 = read_vcf(dir_control3 + "VarScan_SRR1705860_results.vcf")
sample = read_vcf(dir_sample + "VarScan_low_results.vcf")
sample_up = read_vcf(dir_sample + "VarScan_up_results.vcf")

## Выделяем частоту из последней колонки

In [27]:
control1["Sample1"] = pd.to_numeric(control1["Sample1"].str.split(":").str[6].str[:-1].str.replace(",", ".", regex=False))
control2["Sample1"] = pd.to_numeric(control2["Sample1"].str.split(":").str[6].str[:-1].str.replace(",", ".", regex=False))
control3["Sample1"] = pd.to_numeric(control3["Sample1"].str.split(":").str[6].str[:-1].str.replace(",", ".", regex=False))
sample["Sample1"] = pd.to_numeric(sample["Sample1"].str.split(":").str[6].str[:-1].str.replace(",", ".", regex=False))
sample_up["Sample1"] = pd.to_numeric(sample_up["Sample1"].str.split(":").str[6].str[:-1].str.replace(",", ".", regex=False))


## Удаляем не нужное 

In [28]:
control1.drop(columns = ["FORMAT", "INFO", "ID", "QUAL"], inplace=True)
control2.drop(columns = ["FORMAT", "INFO", "ID", "QUAL"], inplace=True)
control3.drop(columns = ["FORMAT", "INFO", "ID", "QUAL"], inplace=True)
sample.drop(columns = ["FORMAT", "INFO", "ID", "QUAL"], inplace=True)
sample_up.drop(columns = ["FORMAT", "INFO", "ID", "QUAL"], inplace=True)

## Переименовываем колонку

In [29]:
control1.rename(columns={'Sample1': 'Frequency'}, inplace=True)
control2.rename(columns={'Sample1': 'Frequency'}, inplace=True)
control3.rename(columns={'Sample1': 'Frequency'}, inplace=True)
sample.rename(columns={'Sample1': 'Frequency'}, inplace=True)
sample_up.rename(columns={'Sample1': 'Frequency'}, inplace=True)



## Ищем среднее и стандартное отклонение 

In [30]:
mean1 = control1["Frequency"].mean()
mean2 = control2["Frequency"].mean()
mean3 = control3["Frequency"].mean()
means = [mean1, mean2, mean3]

In [31]:
std1 = control1["Frequency"].std()
std2 = control2["Frequency"].std()
std3 = control3["Frequency"].std()
stds = [std1, std2, std3]

## Создаем таблицу

In [32]:
table_means = pd.DataFrame({
    "Means": means,
    "Std": stds
})


## Находим пороговое значение для фильтрации

In [33]:
table_means["threshold1"] = table_means["Means"] + 3*table_means["Std"]
table_means["threshold2"] = table_means["Means"] - 3*table_means["Std"]
threshold1 = table_means["threshold1"].mean()
threshold2 = table_means["threshold2"].mean()

## Фильтруем Sample

In [34]:
filtered_sample = sample.query('(Frequency > @threshold1 or Frequency < @threshold2) and Frequency < 1')
filtered_sample
filtered_sample.to_csv('filtered_sample.csv', index=False)

In [36]:
sample_up.to_csv('sample_up.csv', index=False)
sample.to_csv('sample_low.csv', index=False)